In [1]:
# File is to define the metrics to evaluate predictions on

import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import imageio
import os
import time

In [69]:
def calc_dists(lines1, lines2, dist_type):
    x1s = lines1[:, 2]
    y1s = lines1[:, 3]
    x2s = lines2[:, 2]
    y2s = lines2[:, 3]
    if dist_type == "L2":
        return np.mean(np.sqrt(np.square(y2s - y1s) + np.square(x2s - x1s)))
    return np.mean(np.abs(y2s-y1s) + np.abs(x2s-x1s))
    
def extract_trajectory(data, ped_id):
    return data[data[:, 1] == ped_id, :]

def overall_mean(gt, pred, ped_id, dist_type="L2"):
    traj1 = extract_trajectory(gt, ped_id)
    traj2 = extract_trajectory(pred, ped_id)
    dist = calc_dists(traj1, traj2, dist_type)
    return dist

def predicted_mean(gt, pred, ped_id, dist_type="L2"):
    traj1 = extract_trajectory(gt, ped_id)[8:, :] # first 8 are ignored since those are given
    traj2 = extract_trajectory(pred, ped_id)[8:, :]
    dist = calc_dists(traj1, traj2, dist_type)
    return dist

def final_mean(gt, pred, ped_id, dist_type="L2"):
    traj1 = extract_trajectory(gt, ped_id)[-1, :].reshape((1,4)) # only use last one
    traj2 = extract_trajectory(pred, ped_id)[-1, :].reshape((1,4))
    dist = calc_dists(traj1, traj2, dist_type)
    return dist

def get_mean_fn(mean_type):
    if mean_type == "overall":
        return overall_mean
    if mean_type == "predicted":
        return predicted_mean
    return final_mean

# mean_type = overall, predicted, final
# dist_type = L2, L1
def prediction_error(gt, pred, mean_type, dist_type):
    peds = np.sort(np.unique(gt[:, 1]))
    npeds = peds.shape[0]
    dist = 0.0
    for i in range(npeds):
        dist += get_mean_fn(mean_type)(gt, pred, peds[i], dist_type)
    dist /= npeds
    return dist

def file_error(gt, pred):
    return [prediction_error(gt, pred, "overall", "L2"), 
            prediction_error(gt, pred, "predicted", "L2"), 
            prediction_error(gt, pred, "final", "L2"),
            prediction_error(gt, pred, "overall", "L1"),
            prediction_error(gt, pred, "predicted", "L1"),
            prediction_error(gt, pred, "final", "L1")]

def print_leaderboard(gt_dir, predict_dir):
    datasets = {}
    datasets[".overall"] = np.zeros((6,))
    overallcount = 0.0
    for folder in os.listdir(gt_dir):
        if "." in folder:
            continue
        datasets[folder] = {}
        datasets[folder][".err"] = np.zeros((6,))
        count = 0.0
        for fname in os.listdir(gt_dir + folder):
            #print "Processing", fname
            gtfile = gt_dir + folder + "/" + fname
            predfile = predict_dir + folder + "/" + fname
            gt = np.loadtxt(gtfile)
            pred = np.loadtxt(predfile)
            #print gt.shape
            #print pred.shape
            datasets[folder][fname] = file_error(gt, pred)
            datasets[folder][".err"] += np.asarray(datasets[folder][fname])
            datasets[".overall"] += np.asarray(datasets[folder][fname])
            count += 1
            overallcount += 1
        datasets[folder][".err"] /= count
    datasets[".overall"] /= overallcount
    return datasets

In [71]:
datasets = print_leaderboard("../data/challenges/1/gt/", "../data/challenges/1/predict/")

In [77]:
print '{:23} {:5} {:5} {:5} {:5} {:5} {:5}'.format("File", "O-L2", "P-L2", "F-L2", "O-L1", "P-L1", "F-L1")

for folder in datasets:
    if folder == ".overall":
        es = datasets[folder]
        print '{:23} {:5} {:5} {:5} {:5} {:5} {:5}'.format("Overall", es[0], es[1], es[2], es[3], es[4], es[5])
        continue
    for fname in datasets[folder]:
        ffname = "    " + fname if fname != ".err" else folder + "/"
        es = datasets[folder][fname]
        print '{:23} {:5} {:5} {:5} {:5} {:5} {:5}'.format(ffname, es[0], es[1], es[2], es[3], es[4], es[5])

File                    O-L2  P-L2  F-L2  O-L1  P-L1  F-L1 
Overall                 0.499681089934 0.832801816557 1.69097079637 0.618758247428 1.03126374571 2.09952241653
biwi/                   0.607053219715 1.01175536619 1.98538744913 0.766534313725 1.27755718954 2.53284313725
    biwi_eth.txt        0.607053219715 1.01175536619 1.98538744913 0.766534313725 1.27755718954 2.53284313725
    uni_examples.txt    0.50886130961 0.848102182684 1.66553991869 0.649586176471 1.08264362745 2.11061764706
crowds/                 0.445598585022 0.74266430837 1.46544474835 0.568555588235 0.947592647059 1.85966674703
    crowds_zara01.txt   0.382335860434 0.637226434057 1.26534957802 0.487525 0.812541666667 1.60871584699
    little_2.txt        0.434554488841 0.724257481402 1.33579848174 0.521790178571 0.869650297619 1.61119047619
    quad_2.txt          0.196331381519 0.327218969198 0.558936023129 0.249080416667 0.415134027778 0.73305
    little_3.txt        0.502561236433 0.837602060721 1.6134398